# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1tGcX2qzS2GoaN5zFeD5DVJxqDmoQdmMs7QncZwFCEqU/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's totally optional whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_All code should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach._

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [1]:
# all import statements needed for the project, for example:

import math

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import os

In [2]:
# any general notebook setup, like log formatting

In [3]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library
* [ ] Write at least one unit test that tests this distance calculation function. 
* Taxi data:
    * [ ] Use `requests`, BeautifulSoup (`bs4`), and `pandas` to programmatically download the required CSV files & load into memory.
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
    * [ ] Load the data from your local computer (using `pandas`), then clean the data, including: 
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Remove trips that start and/or end outside the designated [coordinate box]
* Using the function that calculates the distance between two coordinates in kilometers, add a column to each `pandas` DataFrame of data that calculates the distance between pickup and dropoff locations for each trip.
* Weather data:
    * [ ] Download the data manually in the link provided in the project doc.
    * [ ] Load the data from your local computer (using `pandas`), then clean the data, including: 
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
        * You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 
* Take a look at the lecture notes from the `pandas` lecture for hints on helpful functionality

### Calculating distance
_Write some prose that tells the reader what you're about to do here._

In [4]:
def calculate_distance(from_coord, to_coord):
    lon1, lat1 = from_coord
    lon2, lat2 = to_coord
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371
    return c * r

In [5]:
def test_calculate_distance():
    assert calculate_distance((1,1),(1,1)) == 0

In [6]:
def add_distance_column(dataframe):
    from_coords = [(x,y) for x,y in zip(dataframe["pickup_longitude"].tolist(),dataframe["pickup_latitude"].tolist())]
    to_coords = [(x,y) for x,y in zip(dataframe["dropoff_longitude"].tolist(),dataframe["dropoff_latitude"].tolist())]
    dataframe["distance"] = [calculate_distance(x,y) for x,y in zip(from_coords,to_coords)]
    return dataframe

### Processing Uber Data

_Write some prose that tells the reader what you're about to do here._

In [7]:
def load_and_clean_uber_data(csv_file):
    uber_rides = pd.read_csv('uber_rides_sample.csv')
    uber_rides = uber_rides.drop(['Unnamed: 0'], axis=1) #Remove unnecessary columns
    uber_rides = uber_rides.dropna(axis=0,how='any')
    uber_rides = uber_rides[(uber_rides['passenger_count'] <= 7) & (uber_rides['passenger_count'] > 0)]
    uber_rides = uber_rides[uber_rides['fare_amount'] <= 100]
    uber_rides = uber_rides[uber_rides['fare_amount'] >= 0]
    uber_rides = uber_rides[uber_rides['pickup_longitude'] != 0]
    uber_rides = uber_rides[uber_rides['pickup_latitude'] != 0]
    uber_rides = uber_rides[uber_rides['dropoff_longitude'] != 0]
    uber_rides = uber_rides[uber_rides['dropoff_latitude'] != 0]
    uber_rides = uber_rides[uber_rides['pickup_longitude'] <= -73.717047]
    uber_rides = uber_rides[uber_rides['pickup_longitude'] >= -74.242330]
    uber_rides = uber_rides[uber_rides['dropoff_longitude'] <= -73.717047]
    uber_rides = uber_rides[uber_rides['dropoff_longitude'] >= -74.242330]
    uber_rides = uber_rides[uber_rides['pickup_latitude'] <= 40.908524]
    uber_rides = uber_rides[uber_rides['pickup_latitude'] >= 40.560445]
    uber_rides = uber_rides[uber_rides['dropoff_latitude'] <= 40.908524]
    uber_rides = uber_rides[uber_rides['dropoff_latitude'] >= 40.560445]
    uber_rides.drop_duplicates(['key'],keep='first',inplace=True)
    return uber_rides
    
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    uber_dataframe = add_distance_column(uber_dataframe)
    return uber_dataframe

### Processing Weather Data

_Write some prose that tells the reader what you're about to do here._

In [8]:
def clean_month_weather_data_hourly(csv_file):
    weather_data = pd.read_csv(csv_file,low_memory=False)
    weather_data = weather_data[["DATE","LATITUDE","LONGITUDE",'HourlyPrecipitation','HourlyWindSpeed']]
    weather_data["DATES"] = weather_data["DATE"]
    del weather_data["DATE"]
    weather_data = weather_data.loc[~pd.isnull(weather_data["HourlyPrecipitation"])].replace("T",0)
    weather_data = weather_data.dropna(subset=['HourlyPrecipitation', 'HourlyWindSpeed'])
    weather_data.drop_duplicates(['DATES'],keep='first',inplace=True)
    return weather_data
    
    
def clean_month_weather_data_daily(csv_file):
    weather_data = pd.read_csv(csv_file,low_memory=False)
    weather_data = weather_data[["DATE","LATITUDE","LONGITUDE",'HourlyPrecipitation','HourlyWindSpeed','DailyAverageWindSpeed','DailyPrecipitation']]
    #weather_data = weather_data.loc[~pd.isnull(weather_data["HourlyPrecipitation"])].replace("T",0)
    weather_data = weather_data.dropna(subset=['HourlyPrecipitation', 'HourlyWindSpeed'])
    weather_data["HourlyPrecipitation"] = weather_data["HourlyPrecipitation"].replace("T",0)
    DailyAverageWindSpeed = weather_data["DailyAverageWindSpeed"].tolist()
    DailyPrecipitation = weather_data["DailyPrecipitation"].tolist()
    DATE = weather_data["DATE"].tolist()
    
    new_AverageWindSpeed = []
    new_Precipitation = []
    new_DATE = []
    LATITUDE = []
    LONGITUDE = []
    
    for idx in range(len(DailyAverageWindSpeed)):
        date = DATE[idx]
        date = date.split("T")[0]
        if date in new_DATE:
            continue
        new_DATE.append(date)
        lon = weather_data.iloc[idx]["LONGITUDE"]
        lat = weather_data.iloc[idx]["LATITUDE"]
        LATITUDE.append(lon)
        LONGITUDE.append(lat)
        
        this_day_data = weather_data.loc[weather_data["DATE"].str.startswith(date)]
        this_day_data = this_day_data.loc[(this_day_data["LONGITUDE"] == lon) & (this_day_data["LATITUDE"] == lat)]
        
        if pd.isnull(DailyAverageWindSpeed[idx]):
            x = this_day_data["HourlyWindSpeed"].tolist()
            x = [float(val) for val in x]
            new_AverageWindSpeed.append(sum(x) / len(this_day_data))
        else:
            new_AverageWindSpeed.append(DailyAverageWindSpeed[idx])
        if pd.isnull(DailyPrecipitation[idx]):
            x = this_day_data["HourlyWindSpeed"].tolist()
            x = [float(val) for val in x]
            new_Precipitation.append(sum(x))
        else:
            new_Precipitation.append(DailyPrecipitation[idx])
    #print(len(new_DATE),len(LATITUDE),len(LONGITUDE),len(new_AverageWindSpeed),len(DailyPrecipitation))
    return pd.DataFrame({
        "DATES":new_DATE,
        "LATITUDE":LATITUDE,
        "LONGITUDE":LONGITUDE,
        "DailyAverageWindSpeed":new_AverageWindSpeed,
        "DailyPrecipitation":new_Precipitation,
    })
    

def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    weather_csv_files = ["2009_weather.csv","2010_weather.csv","2011_weather.csv","2012_weather.csv",\
                         "2013_weather.csv","2014_weather.csv","2015_weather.csv"]

    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    return hourly_data, daily_data

### Processing Taxi Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
def find_taxi_csv_urls():
    content = requests.get(TAXI_URL)
    soup = bs(content.text, 'html.parser')
    tags = soup('a')
    years = ['2009','2010','2011','2012','2013','2014','2015']
    montths = ['01','02','03','04','05','06','07','08','09','10','11','12']
    urls = []
    for tag in tags:
        href = (tag.get('href', None))
        if href.endswith(".csv") and 'yellow_tripdata_' in href:
            if any(ext in href for ext in years):
                urls.append(href)
    return urls

def get_and_clean_month_taxi_data(url):
    df = pd.read_csv(url,skiprows=range(1,30),nrows = 2497)
    return df
        
        
def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls()
    for csv_url in all_csv_urls:
        #print(csv_url)
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month_taxi_data(csv_url)
        #add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_taxi_dataframes.append(dataframe)
        #print(all_taxi_dataframes)
        
    # create one gigantic dataframe with data from every month needed
    df = pd.concat(all_taxi_dataframes)
    df1 = df.loc[~df['pickup_longitude'].isnull()]
    df2 = df.loc[~df['End_Lon'].isnull()]
    df3 = df.loc[~df[' pickup_longitude'].isnull()]
    df1 = df1[['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
           'passenger_count', 'trip_distance', 'pickup_longitude',
           'pickup_latitude',
           'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
           'mta_tax', 'tip_amount', 'tolls_amount',
           'improvement_surcharge', 'total_amount', 'vendor_id',
           'pickup_datetime',
           'dropoff_datetime', 'rate_code', 'surcharge']]
    df2 = df2[['mta_tax', 'surcharge', 'vendor_name',
           'Trip_Pickup_DateTime', 'Trip_Dropoff_DateTime', 'Passenger_Count',
           'Trip_Distance', 'Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat', 'Payment_Type', 'Fare_Amt',
           'Tip_Amt', 'Tolls_Amt', 'Total_Amt']]
    df3 = df3[['vendor_id',
           ' pickup_datetime', ' dropoff_datetime', ' passenger_count',
           ' trip_distance', ' pickup_longitude', ' pickup_latitude',
           ' dropoff_longitude', ' dropoff_latitude',
           ' payment_type', ' fare_amount', ' surcharge', ' mta_tax',
           ' tip_amount', ' tolls_amount', ' total_amount']]
    df2.rename(columns={"Tip_Amt":"tip_amount","Start_Lon": "pickup_longitude", "Start_Lat": "pickup_latitude","End_Lon":"dropoff_longitude","End_Lat":"dropoff_latitude","Passenger_Count":"passenger_count","Tolls_Amt":"tolls_amount","Total_Amt":"total_amount","Trip_Pickup_DateTime":"pickup_datetime","Trip_Dropoff_DateTime":"dropoff_datetime","Trip_Distance":"trip_distance","Payment_Type":"payment_type","Fare_Amt":"fare_amount"},inplace = True)
    df3.rename(columns={" surcharge":"surcharge"," mta_tax":"mta_tax","":""," tip_amount":"tip_amount", " pickup_longitude": "pickup_longitude", " pickup_latitude": "pickup_latitude"," dropoff_longitude":"dropoff_longitude"," dropoff_latitude":"dropoff_latitude"," passenger_count":"passenger_count"," tolls_amount":"tolls_amount"," total_amount":"total_amount"," pickup_datetime":"pickup_datetime"," dropoff_datetime":"dropoff_datetime"," trip_distance":"trip_distance"," payment_type":"payment_type"," fare_amount":"fare_amount"},inplace = True)
    df_all = pd.concat([df1,df2,df3])
    df_11 = df_all[~df_all['tpep_pickup_datetime'].isnull()]
    df_22 = df_all[~df_all['pickup_datetime'].isnull()]
    df_11.drop(columns = ["pickup_datetime","dropoff_datetime"],inplace = True)
    df_11.rename(columns = {"tpep_pickup_datetime":"pickup_datetime","tpep_dropoff_datetime":"dropoff_datetime"},inplace = True)
    df_final = pd.concat([df_22,df_11])
    df_final.drop(columns = ["tpep_pickup_datetime","tpep_dropoff_datetime"],inplace = True)
    df_final['pickup_longitude'] = df_final['pickup_longitude'].replace(np.nan, 0)
    df_final['pickup_latitude'] = df_final['pickup_latitude'].replace(np.nan, 0)
    df_final['dropoff_longitude'] = df_final['dropoff_longitude'].replace(np.nan, 0)
    df_final['dropoff_latitude'] = df_final['dropoff_latitude'].replace(np.nan, 0)
    add_distance_column(df_final)
    return df_final

### Process All Data

_This is where you can actually execute all the required functions._

_Write some prose that tells the reader what you're about to do here._

In [9]:
uber_data = get_uber_data()
hourly_weather, daily_weather = load_and_clean_weather_data()
taxi_data = get_and_clean_taxi_data()

In [10]:
uber_data

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance
0,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1,1.683323
1,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1,2.457590
2,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1,5.036377
3,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3,1.661683
4,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5,4.475450
...,...,...,...,...,...,...,...,...,...
199995,2012-10-28 10:49:00.00000053,3.0,2012-10-28 10:49:00 UTC,-73.987042,40.739367,-73.986525,40.740297,1,0.112210
199996,2014-03-14 01:09:00.0000008,7.5,2014-03-14 01:09:00 UTC,-73.984722,40.736837,-74.006672,40.739620,1,1.875050
199997,2009-06-29 00:42:00.00000078,30.9,2009-06-29 00:42:00 UTC,-73.986017,40.756487,-73.858957,40.692588,2,12.850319
199998,2015-05-20 14:56:25.0000004,14.5,2015-05-20 14:56:25 UTC,-73.997124,40.725452,-73.983215,40.695415,1,3.539715


In [11]:
hourly_weather

,LATITUDE,LONGITUDE,HourlyPrecipitation,HourlyWindSpeed,DATES
36,40.77898,-73.96925,0,14.0,2009-01-02T12:34:00
37,40.77898,-73.96925,0,11.0,2009-01-02T12:51:00
38,40.77898,-73.96925,0,7.0,2009-01-02T13:05:00
39,40.77898,-73.96925,0,9.0,2009-01-02T13:30:00
40,40.77898,-73.96925,0,10.0,2009-01-02T13:46:00
...,...,...,...,...,...
11379,40.77898,-73.96925,0.00,3.0,2015-12-31T18:51:00
11380,40.77898,-73.96925,0.00,6.0,2015-12-31T19:51:00
11381,40.77898,-73.96925,0.00,10.0,2015-12-31T20:51:00
11383,40.77898,-73.96925,0.00,7.0,2015-12-31T22:51:00


In [12]:
daily_weather

,DATES,LATITUDE,LONGITUDE,DailyAverageWindSpeed,DailyPrecipitation
0,2009-01-02,-73.96925,40.77898,9.111111,82.0
1,2009-01-03,-73.96925,40.77898,8.000000,8.0
2,2009-01-05,-73.96925,40.77898,9.000000,9.0
3,2009-01-06,-73.96925,40.77898,9.100000,91.0
4,2009-01-07,-73.96925,40.77898,10.057143,352.0
...,...,...,...,...,...
357,2015-12-27,-73.96925,40.77898,5.666667,136.0
358,2015-12-28,-73.96925,40.77898,8.130435,187.0
359,2015-12-29,-73.96925,40.77898,7.868421,299.0
360,2015-12-30,-73.96925,40.77898,4.035714,113.0


## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [13]:
engine = db.create_engine(DATABASE_URL)

In [14]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS HOURLY_WEATHER (
   DATES TEXT PRIMARY KEY,
   LATITUDE FLOAT,
   LONGITUDE FLOAT,
   HourlyPrecipitation FLOAT,
   HourlyWindSpeed FLOAT
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS DAILY_WEATHER (
   DATES TEXT PRIMARY KEY,
   LATITUDE FLOAT,
   LONGITUDE FLOAT,
   DailyAverageWindSpeed FLOAT,
   DailyPrecipitation FLOAT
);
"""
#need to make sure it is datetime or text and there is two distance in this file
TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS TAXI_TRIPS (
   VendorID TEXT PRIMARY KEY,
   total_amount FLOAT,
   trip_distance FLOAT,
   distance FLOAT
   passenger_count INTEGER,
   pickup_datetime DATETIME,
   dropoff_datetime DATETIME
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS UBER_TRIPS (
   key TEXT PRIMARY KEY,
   pickup_longitude FLOAT,
   pickup_latitude FLOAT,
   dropoff_longitude FLOAT,
   dropoff_latitude FLOAT,
   fare_amount FLOAT,
   distance FLOAT,
   passenger_count INTEGER,
   pickup_datetime TEXT
)
"""

In [15]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [16]:
engine.execute("drop table UBER_TRIPS")
engine.execute("drop table TAXI_TRIPS")
engine.execute("drop table HOURLY_WEATHER")
engine.execute("drop table DAILY_WEATHER")

In [17]:
# create the tables with the schema files
with engine.connect() as connection:
    commands = open(DATABASE_SCHEMA_FILE, "r").read().split(";")
    for command in commands:
        engine.execute(command)

### Add Data to Database

_Write some prose that tells the reader what you're about to do here._

In [18]:
def dataframes_to_table(table_name_to_dataframe):
    for table,dataframe in table_name_to_dataframe.items():
        pd.io.sql.to_sql(dataframe, table, con = engine, index = False, if_exists = 'append')

In [19]:
table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_weather,
    "daily_weather": daily_weather,
}

dataframes_to_table(table_name_to_dataframe)

NameError: name 'taxi_data' is not defined

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [ ]:
def write_query_to_file(query, outfile):
    raise NotImplemented()

### Query N

_Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_N = """
TODO
"""

In [ ]:
engine.execute(QUERY_N).fetchall()

In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.
* [ ] Come up with 3 questions on your own that can be answered based on the data in the 4 tables. Create at least one visualization to answer each question. At least one visualization should require data from at least 3 tables.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_You don't have to query the data directly from the database. You can just re-use the pandas DataFrame that you created in Part 1._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
plot_visual_n(some_dataframe)